# Sudoku Project

For these datasets, I would like to demonstrate LP in these sets.

In [50]:
!pip install cvxopt

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as scs # sparse matrix construction 
import scipy.linalg as scl # linear algebra algorithms
import scipy.optimize as sco # for minimization use
import matplotlib.pylab as plt # for visualization

def fixed_constraints(N=9):
    rowC = np.zeros(N)
    rowC[0] =1
    rowR = np.zeros(N)
    rowR[0] =1
    row = scl.toeplitz(rowC, rowR)
    ROW = np.kron(row, np.kron(np.ones((1,N)), np.eye(N)))
    
    colR = np.kron(np.ones((1,N)), rowC)
    col  = scl.toeplitz(rowC, colR)
    COL  = np.kron(col, np.eye(N))
    
    M = int(np.sqrt(N))
    boxC = np.zeros(M)
    boxC[0]=1
    boxR = np.kron(np.ones((1, M)), boxC) 
    box = scl.toeplitz(boxC, boxR)
    box = np.kron(np.eye(M), box)
    BOX = np.kron(box, np.block([np.eye(N), np.eye(N) ,np.eye(N)]))
    
    cell = np.eye(N**2)
    CELL = np.kron(cell, np.ones((1,N)))
    
    return scs.csr_matrix(np.block([[ROW],[COL],[BOX],[CELL]]))




# For the constraint from clues, we extract the nonzeros from the quiz string.
def clue_constraint(input_quiz, N=9):
    m = np.reshape([int(c) for c in input_quiz], (N,N))
    r, c = np.where(m.T)
    v = np.array([m[c[d],r[d]] for d in range(len(r))])
    
    table = N * c + r
    table = np.block([[table],[v-1]])
    
    # it is faster to use lil_matrix when changing the sparse structure.
    CLUE = scs.lil_matrix((len(table.T), N**3))
    for i in range(len(table.T)):
        CLUE[i,table[0,i]*N + table[1,i]] = 1
    # change back to csr_matrix.
    CLUE = CLUE.tocsr() 
    
    return CLUE

In [7]:
from cvxopt import solvers, matrix
import time
solvers.options['show_progress'] = False

# We test the following algoritm on small data set.
data = pd.read_csv("Desktop/small1.csv") 

corr_cnt = 0
start = time.time()
for i in range(len(data)):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    A0 = fixed_constraints()
    A1 = clue_constraint(quiz)

    # Formulate the matrix A and vector B (B is all ones).
    A = scs.vstack((A0,A1))
    A = A.toarray()
    B = np.ones(A.shape[0])


    # Because rank defficiency. We need to extract effective rank.
    u, s, vh = np.linalg.svd(A, full_matrices=False)
    K = np.sum(s > 1e-12)
    S_ = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
    A = S_@vh
    B = u.T@B
    B = B[:K]

    c = matrix(np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ]))
    G = matrix(np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                         [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]]))
    h = matrix(np.zeros(A.shape[1]*2))
    H = matrix(np.block([A, -A]))
    b = matrix(B)

    sol = solvers.lp(c,G,h,H,b)

    # postprocessing the solution
    X = np.array(sol['x']).T[0]
    x = X[:A.shape[1]] - X[A.shape[1]:]
    
    # map to board
    z = np.reshape(x, (81, 9))
    if np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) ) \
                      - np.reshape([int(c) for c in solu], (9,9)), np.inf) >0:
        pass
    else:
        #print("CORRECT")
        corr_cnt += 1
    
    if (i+1) % 5 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   1.01 secs. Success rate: 5 / 5 
Aver Time:   1.03 secs. Success rate: 10 / 10 
Aver Time:   1.03 secs. Success rate: 15 / 15 
Aver Time:   1.05 secs. Success rate: 20 / 20 
Aver Time:   1.02 secs. Success rate: 24 / 24 


In [8]:
from cvxopt import solvers, matrix
import time
solvers.options['show_progress'] = False

# We test the following algoritm on small data set.
data = pd.read_csv("Desktop/small2.csv") 

corr_cnt = 0
start = time.time()
for i in range(len(data)):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    A0 = fixed_constraints()
    A1 = clue_constraint(quiz)

    # Formulate the matrix A and vector B (B is all ones).
    A = scs.vstack((A0,A1))
    A = A.toarray()
    B = np.ones(A.shape[0])


    # Because rank defficiency. We need to extract effective rank.
    u, s, vh = np.linalg.svd(A, full_matrices=False)
    K = np.sum(s > 1e-12)
    S_ = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
    A = S_@vh
    B = u.T@B
    B = B[:K]

    c = matrix(np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ]))
    G = matrix(np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                         [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]]))
    h = matrix(np.zeros(A.shape[1]*2))
    H = matrix(np.block([A, -A]))
    b = matrix(B)

    sol = solvers.lp(c,G,h,H,b)

    # postprocessing the solution
    X = np.array(sol['x']).T[0]
    x = X[:A.shape[1]] - X[A.shape[1]:]
    
    # map to board
    z = np.reshape(x, (81, 9))
    if np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) ) \
                      - np.reshape([int(c) for c in solu], (9,9)), np.inf) >0:
        pass
    else:
        #print("CORRECT")
        corr_cnt += 1
    
    if (i+1) % 5 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   1.04 secs. Success rate: 3 / 5 
Aver Time:   1.06 secs. Success rate: 6 / 10 
Aver Time:   1.02 secs. Success rate: 9 / 15 
Aver Time:   1.01 secs. Success rate: 13 / 20 
Aver Time:   1.04 secs. Success rate: 16 / 25 
Aver Time:   1.03 secs. Success rate: 19 / 30 
Aver Time:   1.01 secs. Success rate: 23 / 35 
Aver Time:   1.00 secs. Success rate: 25 / 40 
Aver Time:   1.00 secs. Success rate: 28 / 45 
Aver Time:   1.00 secs. Success rate: 31 / 50 
Aver Time:   1.01 secs. Success rate: 35 / 55 
Aver Time:   1.04 secs. Success rate: 39 / 60 
Aver Time:   1.06 secs. Success rate: 44 / 65 
Aver Time:   1.07 secs. Success rate: 48 / 70 
Aver Time:   1.07 secs. Success rate: 49 / 75 
Aver Time:   1.07 secs. Success rate: 52 / 80 
Aver Time:   1.06 secs. Success rate: 54 / 85 
Aver Time:   1.06 secs. Success rate: 56 / 90 
Aver Time:   1.06 secs. Success rate: 60 / 95 
Aver Time:   1.08 secs. Success rate: 63 / 100 
Aver Time:   1.10 secs. Success rate: 67 / 105 
Aver Time:   1.

Aver Time:   1.03 secs. Success rate: 304 / 850 
Aver Time:   1.03 secs. Success rate: 305 / 855 
Aver Time:   1.03 secs. Success rate: 306 / 860 
Aver Time:   1.03 secs. Success rate: 306 / 865 
Aver Time:   1.03 secs. Success rate: 307 / 870 
Aver Time:   1.03 secs. Success rate: 308 / 875 
Aver Time:   1.03 secs. Success rate: 309 / 880 
Aver Time:   1.03 secs. Success rate: 310 / 885 
Aver Time:   1.03 secs. Success rate: 311 / 890 
Aver Time:   1.03 secs. Success rate: 311 / 895 
Aver Time:   1.03 secs. Success rate: 312 / 900 
Aver Time:   1.03 secs. Success rate: 312 / 905 
Aver Time:   1.03 secs. Success rate: 312 / 910 
Aver Time:   1.03 secs. Success rate: 312 / 915 
Aver Time:   1.02 secs. Success rate: 315 / 920 
Aver Time:   1.02 secs. Success rate: 315 / 925 
Aver Time:   1.02 secs. Success rate: 316 / 930 
Aver Time:   1.02 secs. Success rate: 316 / 935 
Aver Time:   1.02 secs. Success rate: 318 / 940 
Aver Time:   1.02 secs. Success rate: 319 / 945 
Aver Time:   1.02 se

And for the B Datasets, we should use random number seed 42, generate a uniform distributed 1000 random samples from the large sets.

In [9]:
import time

# We test the following algoritm on small data set.
data = pd.read_csv("Downloads/large1.csv") 

corr_cnt = 0
start = time.time()

random_seed = 42
np.random.seed(random_seed)

if len(data) > 1000:
    samples = np.random.choice(len(data), 1000)
else:
    samples = range(len(data))

for i in range(len(samples)):
    quiz = data["quizzes"][samples[i]]
    solu = data["solutions"][samples[i]]
    A0 = fixed_constraints()
    A1 = clue_constraint(quiz)

    # Formulate the matrix A and vector B (B is all ones).
    A = scs.vstack((A0,A1))
    A = A.toarray()
    B = np.ones(A.shape[0])


    # Because rank defficiency. We need to extract effective rank.
    u, s, vh = np.linalg.svd(A, full_matrices=False)
    K = np.sum(s > 1e-12)
    S = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
    A = S@vh
    B = u.T@B
    B = B[:K]

    c = np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ])
    G = np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                         [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]])
    h = np.zeros(A.shape[1]*2)
    H = np.block([A, -A])
    b = B

    ret = sco.linprog(c, G, h, H, b, method='interior-point', options={'tol':1e-6})
    x = ret.x[:A.shape[1]] - ret.x[A.shape[1]:]

    
    z = np.reshape(x, (81, 9))
    if np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) ) \
                      - np.reshape([int(c) for c in solu], (9,9)), np.inf) >0:
        pass
    else:
        #print("CORRECT")
        corr_cnt += 1

    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   0.25 secs. Success rate: 16 / 20 
Aver Time:   0.21 secs. Success rate: 32 / 40 
Aver Time:   0.21 secs. Success rate: 47 / 60 
Aver Time:   0.21 secs. Success rate: 64 / 80 
Aver Time:   0.20 secs. Success rate: 78 / 100 
Aver Time:   0.21 secs. Success rate: 92 / 120 
Aver Time:   0.21 secs. Success rate: 107 / 140 
Aver Time:   0.20 secs. Success rate: 124 / 160 
Aver Time:   0.20 secs. Success rate: 138 / 180 
Aver Time:   0.20 secs. Success rate: 155 / 200 
Aver Time:   0.21 secs. Success rate: 173 / 220 
Aver Time:   0.21 secs. Success rate: 191 / 240 
Aver Time:   0.20 secs. Success rate: 208 / 260 
Aver Time:   0.20 secs. Success rate: 224 / 280 
Aver Time:   0.20 secs. Success rate: 242 / 300 
Aver Time:   0.20 secs. Success rate: 257 / 320 
Aver Time:   0.20 secs. Success rate: 275 / 340 
Aver Time:   0.20 secs. Success rate: 291 / 360 
Aver Time:   0.20 secs. Success rate: 308 / 380 
Aver Time:   0.20 secs. Success rate: 328 / 400 
Aver Time:   0.20 secs. Succes

In [10]:
import time
data = pd.read_csv("Downloads/large2.csv") 

corr_cnt = 0
start = time.time()

random_seed = 42
np.random.seed(random_seed)

if len(data) > 1000:
    samples = np.random.choice(len(data), 1000)
else:
    samples = range(len(data))

for i in range(len(samples)):
    quiz = data["quizzes"][samples[i]]
    solu = data["solutions"][samples[i]]
    A0 = fixed_constraints()
    A1 = clue_constraint(quiz)

    # Formulate the matrix A and vector B (B is all ones).
    A = scs.vstack((A0,A1))
    A = A.toarray()
    B = np.ones(A.shape[0])


    # Because rank defficiency. We need to extract effective rank.
    u, s, vh = np.linalg.svd(A, full_matrices=False)
    K = np.sum(s > 1e-12)
    S = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
    A = S@vh
    B = u.T@B
    B = B[:K]

    c = np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ])
    G = np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                         [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]])
    h = np.zeros(A.shape[1]*2)
    H = np.block([A, -A])
    b = B

    ret = sco.linprog(c, G, h, H, b, method='interior-point', options={'tol':1e-6})
    x = ret.x[:A.shape[1]] - ret.x[A.shape[1]:]

    
    z = np.reshape(x, (81, 9))
    if np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) ) \
                      - np.reshape([int(c) for c in solu], (9,9)), np.inf) >0:
        pass
    else:
        #print("CORRECT")
        corr_cnt += 1

    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   0.19 secs. Success rate: 20 / 20 
Aver Time:   0.18 secs. Success rate: 40 / 40 
Aver Time:   0.18 secs. Success rate: 60 / 60 
Aver Time:   0.18 secs. Success rate: 80 / 80 
Aver Time:   0.18 secs. Success rate: 100 / 100 
Aver Time:   0.18 secs. Success rate: 120 / 120 
Aver Time:   0.18 secs. Success rate: 140 / 140 
Aver Time:   0.19 secs. Success rate: 160 / 160 
Aver Time:   0.18 secs. Success rate: 180 / 180 
Aver Time:   0.18 secs. Success rate: 200 / 200 
Aver Time:   0.18 secs. Success rate: 220 / 220 
Aver Time:   0.19 secs. Success rate: 240 / 240 
Aver Time:   0.19 secs. Success rate: 260 / 260 
Aver Time:   0.19 secs. Success rate: 280 / 280 
Aver Time:   0.18 secs. Success rate: 300 / 300 
Aver Time:   0.18 secs. Success rate: 320 / 320 
Aver Time:   0.18 secs. Success rate: 340 / 340 
Aver Time:   0.18 secs. Success rate: 360 / 360 
Aver Time:   0.18 secs. Success rate: 380 / 380 
Aver Time:   0.18 secs. Success rate: 400 / 400 
Aver Time:   0.18 secs. Succ